In [1]:
import gc
import os
import psutil
import IPython

# ✅ Function to Monitor RAM Usage
def check_ram():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3  # Convert to GB
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

# ✅ Function to Clean RAM
def clean_ram():
    gc.collect()  # ✅ Free Unused Memory
    IPython.display.clear_output(wait=True)  # ✅ Clear Output (Optional)
    check_ram()  # ✅ Show Updated RAM Usage

# Example Usage:
# Run this after every large operation (like merging datasets, model training)
check_ram()  # Before cleaning
clean_ram()  # After cleaning

🔍 Current RAM Usage: 0.07 GB


In [2]:
import pandas as pd

# Load the cleaned dataset
file_path = "dataset/working1/cleaned_dataset.csv"  # Change this path if needed
df = pd.read_csv(file_path)

# Get the feature names (excluding the label column)
feature_names = df.columns.tolist()
feature_names.remove("Label")

print("✅ Features Used for Training:")
print(feature_names)

# Save feature names to a text file (to transfer to Ubuntu VM)
with open("dataset/working1/feature_list.txt", "w") as f:
    for feature in feature_names:
        f.write(feature + "\n")

print("\n✅ Feature names saved to feature_list.txt")

✅ Features Used for Training:
['Dst Port', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Wi

NORMALIZE DATA AND SPLIT DATASET

In [ ]:
import pandas as pd
import numpy as np
import gc
import joblib # Để lưu và tải scaler
import os # Để làm việc với đường dẫn file

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Import các phương pháp xử lý mất cân bằng từ imblearn
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler

# --- Thiết lập chung ---
cleaned_file_path = "/content/drive/MyDrive/ai_model/cleaned_dataset.csv"
output_data_dir = "/content/drive/MyDrive/ai_model/working2"
os.makedirs(output_data_dir, exist_ok=True)
scaler_path = os.path.join(output_data_dir, "scaler.pkl")
test_data_output_path = os.path.join(output_data_dir, "test1.csv")

# --- 1. Tải Dữ liệu Đã Làm Sạch ---
print(f"📂 Đang tải Dataset từ: {cleaned_file_path} ...")
try:
    df = pd.read_csv(cleaned_file_path)
    print(f"✅ Dataset đã tải! Shape: {df.shape}")
except FileNotFoundError:
    print(f"❌ LỖI: Không tìm thấy file '{cleaned_file_path}'. Vui lòng kiểm tra lại đường dẫn.")
    exit()
except Exception as e:
    print(f"❌ LỖI không xác định khi tải dữ liệu: {e}")
    exit()

# --- 2. Mã hóa Cột Nhãn ---
label_mapping = {
    "Benign": 0, "Bot": 1, "DDOS attack-HOIC": 2, "DDOS attack-LOIC-UDP": 3,
    "DoS attacks-GoldenEye": 4, "DoS attacks-Hulk": 5, "DoS attacks-SlowHTTPTest": 6,
    "DoS attacks-Slowloris": 7, "FTP-BruteForce": 8, "Infilteration": 9, # Có thể là "Infiltration"
    "SSH-Bruteforce": 10, "Brute Force -Web": 11, "Brute Force -XSS": 12, "SQL Injection": 13
}
if "Label" not in df.columns:
    print(f"❌ LỖI: Không tìm thấy cột 'Label' trong DataFrame. Các cột hiện có: {df.columns.tolist()}")
    exit()
df["Label"] = df["Label"].map(label_mapping)
if df["Label"].isnull().any():
    print("\n⚠️ CẢNH BÁO: Có giá trị NaN trong cột 'Label' sau khi map. Các hàng này sẽ bị loại bỏ.")
    df.dropna(subset=["Label"], inplace=True)
df["Label"] = df["Label"].astype(int)
print("\n✅ Cột 'Label' đã được mã hóa sang dạng số.")
print("Phân bố các lớp sau khi mã hóa và loại bỏ NaN (nếu có):")
print(df["Label"].value_counts().sort_index())

# --- 3. Chia Dữ liệu Train-Test ---
print("\n🔄 Đang chia dữ liệu thành tập huấn luyện và tập kiểm thử...")
if df.empty or "Label" not in df.columns:
    print("❌ LỖI: DataFrame rỗng hoặc thiếu cột 'Label' trước khi chia train-test.")
    exit()

# Kiểm tra xem tất cả các lớp có đủ mẫu để stratify không
label_counts_for_split = df["Label"].value_counts()
if (label_counts_for_split < 2).any() and (label_counts_for_split < (1/0.2)): # 0.2 là test_size
    print(f"⚠️ CẢNH BÁO: Một số lớp có ít hơn 2 mẫu hoặc không đủ mẫu cho stratify với test_size=0.2: \n{label_counts_for_split[label_counts_for_split < 2]}")
    print("   Cân nhắc việc gộp các lớp hiếm này hoặc loại bỏ chúng nếu không thể stratify.")
    # Hoặc thử không dùng stratify nếu bắt buộc, nhưng không khuyến khích cho dữ liệu mất cân bằng
    # X = df.drop(columns=["Label"])
    # y = df["Label"]
    # X_train_df, X_test_df, y_train_series, y_test_series = train_test_split(X, y, test_size=0.2, random_state=20) # Bỏ stratify
    # print("   Đã thử chia không dùng stratify.")
    # Tạm thời dừng nếu có lớp quá ít mẫu gây lỗi stratify
    if (label_counts_for_split < 2).any():
         print("   Một số lớp có ít hơn 2 mẫu, không thể stratify. Dừng chương trình.")
         exit()


try:
    X = df.drop(columns=["Label"])
    y = df["Label"]
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=20, stratify=y)
    X_train_df = df_train.drop(columns=["Label"])
    y_train_series = df_train["Label"]
    X_test_df = df_test.drop(columns=["Label"])
    y_test_series = df_test["Label"]

    print(f"✅ Chia dữ liệu hoàn tất!")
    print(f"   Kích thước tập huấn luyện (df_train): {df_train.shape}")
    print(f"   Kích thước tập kiểm thử (df_test): {df_test.shape}")
except ValueError as e_split:
    print(f"❌ LỖI khi chia train-test (có thể do stratify với lớp quá ít mẫu): {e_split}")
    exit()

# --- 4. Chuẩn hóa Dữ liệu Số học ---
print("\n🔄 Đang chuẩn hóa các cột số học...")
numeric_cols = X_train_df.select_dtypes(include=np.number).columns.tolist()
if not numeric_cols:
    print("❌ LỖI: Không tìm thấy cột số học nào trong X_train_df để chuẩn hóa.")
    exit()
scaler = MinMaxScaler()
scaler.fit(X_train_df[numeric_cols])
X_train_scaled_df = X_train_df.copy()
X_test_scaled_df = X_test_df.copy()
X_train_scaled_df[numeric_cols] = scaler.transform(X_train_df[numeric_cols])
X_test_scaled_df[numeric_cols] = scaler.transform(X_test_df[numeric_cols])
print("✅ Chuẩn hóa hoàn tất!")
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler đã được lưu tại: {scaler_path}")
df_train_scaled = X_train_scaled_df.copy()
df_train_scaled["Label"] = y_train_series.values
df_test_scaled = X_test_scaled_df.copy()
df_test_scaled["Label"] = y_test_series.values

# --- 5. Xử lý Mất cân bằng Dữ liệu ---
X_train_features_np = df_train_scaled.drop(columns=["Label"]).values
y_train_labels_np = df_train_scaled["Label"].values

# --- TRỰC QUAN HÓA PHÂN PHỐI LỚP TRƯỚC VÀ SAU KHI CÂN BẰNG DỮ LIỆU ---
print("\n📊 Đang tạo biểu đồ trực quan hóa phân phối lớp...")

# Giả định rằng 'label_mapping' đã được định nghĩa ở phần trước của script của bạn
# Ví dụ: label_mapping = { "Benign": 0, "Bot": 1, ... }
# Tạo inverse_label_mapping để lấy tên lớp cho biểu đồ
if 'label_mapping' in locals() or 'label_mapping' in globals():
    inverse_label_mapping = {v: k for k, v in label_mapping.items()}
else:
    print("⚠️ CẢNH BÁO: Biến 'label_mapping' không được tìm thấy. Biểu đồ sẽ sử dụng nhãn số.")
    # Tạo một ánh xạ giả nếu không có, để code không bị lỗi
    # Lấy tất cả các nhãn duy nhất từ cả hai tập dữ liệu để tạo ánh xạ giả
    unique_labels_before = pd.Series(y_train_labels_np).unique()
    unique_labels_after = df_train_balanced["Label"].unique()
    all_unique_labels = sorted(list(set(list(unique_labels_before) + list(unique_labels_after))))
    inverse_label_mapping = {label: f"Lớp {label}" for label in all_unique_labels}


# 1. Trực quan hóa Phân phối Lớp TRƯỚC khi áp dụng SMOTE (trên tập huấn luyện gốc)
#    'y_train_labels_np' là Series/array chứa nhãn của tập huấn luyện gốc (đã mã hóa số)
train_label_counts_before_smote = pd.Series(y_train_labels_np).value_counts().sort_index()
train_label_names_before_smote = train_label_counts_before_smote.index.map(inverse_label_mapping)

plt.figure(figsize=(14, 8)) # Kích thước biểu đồ
sns.barplot(x=train_label_names_before_smote, y=train_label_counts_before_smote.values, palette="viridis")
plt.title('Phân phối các lớp trong tập Huấn luyện TRƯỚC khi Xử lý Mất cân bằng', fontsize=16)
plt.xlabel('Loại Tấn Công / Benign', fontsize=14)
plt.ylabel('Số lượng Mẫu', fontsize=14)
plt.xticks(rotation=45, ha="right", fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
# Lưu biểu đồ (tùy chọn)
path_before_smote_plot = os.path.join(output_data_dir, f"distribution_before_balancing_{balancing_method_choice}.png")
plt.savefig(path_before_smote_plot, dpi=150)
print(f"✅ Biểu đồ TRƯỚC khi cân bằng đã lưu tại: {path_before_smote_plot}")
plt.show()
print("\n🔍 Phân bố nhãn trong Tập Huấn luyện TRƯỚC khi xử lý mất cân bằng:")
current_class_counts_train = pd.Series(y_train_labels_np).value_counts().sort_index()
print(current_class_counts_train)

balancing_method_choice = 'enhanced_smote'
X_resampled_np = X_train_features_np.copy()
y_resampled_np = y_train_labels_np.copy()
print(f"\n🚀 Áp dụng phương pháp xử lý mất cân bằng: {balancing_method_choice}...")

# Tham số k_neighbors chung cho các phương pháp dựa trên SMOTE
# SMOTE yêu cầu số mẫu của một lớp phải > k_neighbors để có thể oversample lớp đó.
# Nếu một lớp có số mẫu <= k_neighbors, nó sẽ không được SMOTE.
DEFAULT_K_NEIGHBORS = 5

if balancing_method_choice == 'enhanced_smote':
    from imblearn.over_sampling import SMOTE
    target_samples_smote = {}
    minority_classes_to_boost = [3, 6, 8, 9, 11, 12, 13]
    desired_count_for_minorities = 25000

    for cls_label in minority_classes_to_boost:
        num_samples_class = current_class_counts_train.get(cls_label, 0)
        if num_samples_class > 0 and num_samples_class < desired_count_for_minorities:
            if num_samples_class > DEFAULT_K_NEIGHBORS: # Đủ mẫu để SMOTE với k_neighbors hiện tại
                target_samples_smote[cls_label] = desired_count_for_minorities
            else:
                print(f"Lưu ý (SMOTE): Lớp {cls_label} có {num_samples_class} mẫu, không đủ cho k_neighbors={DEFAULT_K_NEIGHBORS}. "
                      f"Lớp này sẽ không được oversample bởi SMOTE với k_neighbors này.")

    if not target_samples_smote:
        print("Không có lớp nào phù hợp để SMOTE theo chiến lược hiện tại. Dữ liệu giữ nguyên.")
    else:
        print(f"Chiến lược SMOTE (số mẫu mục tiêu): {target_samples_smote}")
        smote_sampler = SMOTE(sampling_strategy=target_samples_smote, random_state=42, k_neighbors=DEFAULT_K_NEIGHBORS)
        X_resampled_np, y_resampled_np = smote_sampler.fit_resample(X_train_features_np, y_train_labels_np)

elif balancing_method_choice == 'adasyn':
    from imblearn.over_sampling import ADASYN
    target_samples_adasyn = {}
    minority_classes_to_boost = [3, 6, 8, 9, 11, 12, 13]
    desired_count_for_minorities = 25000

    for cls_label in minority_classes_to_boost:
        num_samples_class = current_class_counts_train.get(cls_label, 0)
        if num_samples_class > 0 and num_samples_class < desired_count_for_minorities:
            # ADASYN cũng dựa trên k-NN (tham số n_neighbors), tương tự k_neighbors của SMOTE
            if num_samples_class > DEFAULT_K_NEIGHBORS: # ADASYN mặc định n_neighbors=5
                target_samples_adasyn[cls_label] = desired_count_for_minorities
            else:
                print(f"Lưu ý (ADASYN): Lớp {cls_label} có {num_samples_class} mẫu, không đủ cho n_neighbors={DEFAULT_K_NEIGHBORS}. "
                      f"Lớp này sẽ không được oversample bởi ADASYN với n_neighbors này.")

    if not target_samples_adasyn:
        print("Không có lớp nào phù hợp để ADASYN. Dữ liệu giữ nguyên.")
    else:
        print(f"Chiến lược ADASYN (số mẫu mục tiêu): {target_samples_adasyn}")
        adasyn_sampler = ADASYN(sampling_strategy=target_samples_adasyn, random_state=42, n_neighbors=DEFAULT_K_NEIGHBORS)
        X_resampled_np, y_resampled_np = adasyn_sampler.fit_resample(X_train_features_np, y_train_labels_np)

elif balancing_method_choice == 'borderline_smote':
    from imblearn.over_sampling import BorderlineSMOTE
    target_samples_bsmote = {}
    minority_classes_to_boost = [3, 6, 8, 9, 11, 12, 13]
    desired_count_for_minorities = 25000

    for cls_label in minority_classes_to_boost:
        num_samples_class = current_class_counts_train.get(cls_label, 0)
        if num_samples_class > 0 and num_samples_class < desired_count_for_minorities:
            # BorderlineSMOTE có k_neighbors (cho SMOTE) và m_neighbors (để xác định borderline)
            # Cả hai đều cần số mẫu > giá trị của chúng. Giữ mốc DEFAULT_K_NEIGHBORS cho cả hai.
            if num_samples_class > DEFAULT_K_NEIGHBORS :
                target_samples_bsmote[cls_label] = desired_count_for_minorities
            else:
                print(f"Lưu ý (BorderlineSMOTE): Lớp {cls_label} có {num_samples_class} mẫu, không đủ cho k_neighbors/m_neighbors={DEFAULT_K_NEIGHBORS}. "
                      f"Lớp này sẽ không được oversample.")
    if not target_samples_bsmote:
        print("Không có lớp nào phù hợp để BorderlineSMOTE. Dữ liệu giữ nguyên.")
    else:
        print(f"Chiến lược BorderlineSMOTE (số mẫu mục tiêu): {target_samples_bsmote}")
        bsmote_sampler = BorderlineSMOTE(sampling_strategy=target_samples_bsmote, random_state=42,
                                         k_neighbors=DEFAULT_K_NEIGHBORS, m_neighbors=min(10, DEFAULT_K_NEIGHBORS*2), kind='borderline-1')
        X_resampled_np, y_resampled_np = bsmote_sampler.fit_resample(X_train_features_np, y_train_labels_np)

elif balancing_method_choice in ['smoteenn', 'smotetomek']:
    # Chiến lược SMOTE bên trong cho các phương pháp kết hợp
    smote_strategy_for_combine = {}
    minority_classes_to_boost_combine = [3, 6, 8, 9, 11, 12, 13]
    desired_count_for_minorities_combine = 30000

    for cls_label in minority_classes_to_boost_combine:
        num_samples_class = current_class_counts_train.get(cls_label, 0)
        if num_samples_class > 0 and num_samples_class < desired_count_for_minorities_combine:
            if num_samples_class > DEFAULT_K_NEIGHBORS:
                 smote_strategy_for_combine[cls_label] = desired_count_for_minorities_combine
            else:
                print(f"Lưu ý ({balancing_method_choice}): Lớp {cls_label} có {num_samples_class} mẫu, không đủ cho SMOTE bên trong với k_neighbors={DEFAULT_K_NEIGHBORS}. "
                      f"Lớp này sẽ không được SMOTE trong bước đầu của {balancing_method_choice}.")

    if not smote_strategy_for_combine:
        print(f"Không có lớp nào phù hợp cho bước SMOTE của {balancing_method_choice}. Dữ liệu giữ nguyên.")
    else:
        print(f"Chiến lược SMOTE cho {balancing_method_choice}: {smote_strategy_for_combine}")
        if balancing_method_choice == 'smoteenn':
            from imblearn.combine import SMOTEENN
            combine_sampler = SMOTEENN(sampling_strategy=smote_strategy_for_combine, random_state=42, smote=SMOTE(k_neighbors=DEFAULT_K_NEIGHBORS, random_state=42))
        else: # smotetomek
            from imblearn.combine import SMOTETomek
            combine_sampler = SMOTETomek(sampling_strategy=smote_strategy_for_combine, random_state=42, smote=SMOTE(k_neighbors=DEFAULT_K_NEIGHBORS, random_state=42))
        X_resampled_np, y_resampled_np = combine_sampler.fit_resample(X_train_features_np, y_train_labels_np)

elif balancing_method_choice == 'random_undersampler':
    from imblearn.under_sampling import RandomUnderSampler
    rus_strategy = {}
    class_counts_sorted_rus = current_class_counts_train.sort_values(ascending=False)
    if len(class_counts_sorted_rus) > 1:
        majority_class_label = class_counts_sorted_rus.index[0]
        second_majority_count = class_counts_sorted_rus.iloc[1]
        target_majority_count = max(int(second_majority_count * 3), 500000)
        if current_class_counts_train.get(majority_class_label, 0) > target_majority_count:
             rus_strategy[majority_class_label] = target_majority_count
    if not rus_strategy:
        print("Không cần Random Undersampling. Dữ liệu giữ nguyên.")
    else:
        print(f"Chiến lược RandomUnderSampler: {rus_strategy}")
        rus_sampler = RandomUnderSampler(sampling_strategy=rus_strategy, random_state=42)
        X_resampled_np, y_resampled_np = rus_sampler.fit_resample(X_train_features_np, y_train_labels_np)

elif balancing_method_choice == 'none':
    print("Không áp dụng phương pháp resampling nào.")
else:
    print(f"Lựa chọn phương pháp '{balancing_method_choice}' không hợp lệ. Dữ liệu giữ nguyên.")

print(f"\n✅ Xử lý mất cân bằng bằng '{balancing_method_choice}' hoàn tất!")
print(f"   Kích thước tập huấn luyện SAU khi resample: X={X_resampled_np.shape}, y={y_resampled_np.shape}")

df_train_balanced = pd.DataFrame(X_resampled_np, columns=X_train_df.columns) # Sử dụng tên cột từ X_train_df
df_train_balanced["Label"] = y_resampled_np

print("\n🔍 Phân bố nhãn trong Tập Huấn luyện SAU khi xử lý mất cân bằng:")
print(df_train_balanced["Label"].value_counts().sort_index())
if X_resampled_np.size > 0 :
    min_val_resampled = df_train_balanced.iloc[:, :-1].values.min()
    max_val_resampled = df_train_balanced.iloc[:, :-1].values.max()
    print(f"   Phạm vi đặc trưng sau khi resample: {min_val_resampled:.4f} đến {max_val_resampled:.4f}")

# --- 6. Lưu Dữ liệu Đã Xử lý ---
output_train_balanced_file_path = os.path.join(output_data_dir, f"train_balanced_by_{balancing_method_choice}.csv")
try:
    df_train_balanced.to_csv(output_train_balanced_file_path, index=False)
    print(f"\n✅ Dữ liệu huấn luyện (đã cân bằng) đã được lưu tại: {output_train_balanced_file_path}")
except Exception as e:
    print(f"❌ LỖI khi lưu file huấn luyện đã cân bằng: {e}")
try:
    df_test_scaled.to_csv(test_data_output_path, index=False)
    print(f"✅ Dữ liệu kiểm thử (đã chuẩn hóa) đã được lưu tại: {test_data_output_path}")
except Exception as e:
    print(f"❌ LỖI khi lưu file kiểm thử: {e}")

# 2. Trực quan hóa Phân phối Lớp SAU khi áp dụng SMOTE (trên tập huấn luyện đã cân bằng)
#    'df_train_balanced' là DataFrame chứa tập huấn luyện sau khi áp dụng kỹ thuật cân bằng
if not df_train_balanced.empty:
    train_label_counts_after_smote = df_train_balanced['Label'].value_counts().sort_index()
    train_label_names_after_smote = train_label_counts_after_smote.index.map(inverse_label_mapping)

    plt.figure(figsize=(14, 8)) # Kích thước biểu đồ
    sns.barplot(x=train_label_names_after_smote, y=train_label_counts_after_smote.values, palette="magma")
    plt.title(f'Phân phối các lớp trong tập Huấn luyện SAU khi áp dụng {balancing_method_choice.upper()}', fontsize=16)
    plt.xlabel('Loại Tấn Công / Benign', fontsize=14)
    plt.ylabel('Số lượng Mẫu (Sau Resampling)', fontsize=14)
    plt.xticks(rotation=45, ha="right", fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    # Lưu biểu đồ (tùy chọn)
    path_after_smote_plot = os.path.join(output_data_dir, f"distribution_after_balancing_{balancing_method_choice}.png")
    plt.savefig(path_after_smote_plot, dpi=150)
    print(f"✅ Biểu đồ SAU khi cân bằng đã lưu tại: {path_after_smote_plot}")
    plt.show()
else:
    print("⚠️ DataFrame 'df_train_balanced' rỗng, không thể tạo biểu đồ sau khi cân bằng.")

print("✅ Hoàn tất trực quan hóa phân phối lớp.")

# --- 7. Xác minh lại Phân bố Nhãn ---
print("\n🔍 Phân bố nhãn trong Tập Huấn luyện đã lưu (df_train_balanced):")
print(df_train_balanced["Label"].value_counts().sort_index())
print("\n🔍 Phân bố nhãn trong Tập Kiểm thử đã lưu (df_test_scaled):")
print(df_test_scaled["Label"].value_counts().sort_index())

# --- 8. Dọn dẹp Bộ nhớ ---
del df, X, y, df_train, df_test
del X_train_df, y_train_series, X_test_df, y_test_series
del X_train_scaled_df, X_test_scaled_df, df_train_scaled
del X_train_features_np, y_train_labels_np
del X_resampled_np, y_resampled_np, df_train_balanced, df_test_scaled
gc.collect()
print("\n✅ Dọn dẹp bộ nhớ hoàn tất.")

📂 Đang tải Dataset từ: dataset/working2/cleaned_dataset.csv ...
✅ Dataset đã tải! Shape: (5881823, 73)

✅ Cột 'Label' đã được mã hóa sang dạng số.
Phân bố các lớp sau khi mã hóa và loại bỏ NaN (nếu có):
Label
0     5087280
1      144535
2      217401
3        1730
4       41406
5      145199
6          55
7        9908
8          53
9      139341
10      94048
11        555
12        228
13         84
Name: count, dtype: int64

🔄 Đang chia dữ liệu thành tập huấn luyện và tập kiểm thử...
✅ Chia dữ liệu hoàn tất!
   Kích thước tập huấn luyện (df_train): (4705458, 73)
   Kích thước tập kiểm thử (df_test): (1176365, 73)

🔄 Đang chuẩn hóa các cột số học...
✅ Chuẩn hóa hoàn tất!
✅ Scaler đã được lưu tại: dataset/working2\scaler.pkl

🔍 Phân bố nhãn trong Tập Huấn luyện TRƯỚC khi xử lý mất cân bằng:
0     4069824
1      115628
2      173921
3        1384
4       33125
5      116159
6          44
7        7926
8          43
9      111473
10      75238
11        444
12        182
13         67
N

In [5]:
import torch
label_mapping = {
    "Benign": 0,
    "Bot": 1,
    "DDOS attack-HOIC": 2,
    "DDOS attack-LOIC-UDP": 3,
    "DoS attacks-GoldenEye": 4,
    "DoS attacks-Hulk": 5,
    "DoS attacks-SlowHTTPTest": 6,
    "DoS attacks-Slowloris": 7,
    "FTP-BruteForce": 8,
    "Infilteration": 9,
    "SSH-Bruteforce": 10,
    "Brute Force -Web": 11,
    "Brute Force -XSS": 12,
    "SQL Injection": 13
}
torch.save(label_mapping, "dataset/working1/label_mapping.pth")

# MODEL TRAINING

In [6]:
import gc
import torch

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Clear Python garbage collection
gc.collect()

print("✅ RAM and GPU memory cleared!")

✅ RAM and GPU memory cleared!


In [7]:
import gc
import os
import psutil
import IPython

# ✅ Function to Monitor RAM Usage
def check_ram():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3  # Convert to GB
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

# ✅ Function to Clean RAM
def clean_ram():
    gc.collect()  # ✅ Free Unused Memory
    IPython.display.clear_output(wait=True)  # ✅ Clear Output (Optional)
    check_ram()  # ✅ Show Updated RAM Usage

# Example Usage:
# Run this after every large operation (like merging datasets, model training)
check_ram()  # Before cleaning
clean_ram()  # After cleaning

🔍 Current RAM Usage: 0.51 GB


In [1]:
import pandas as pd
import numpy as np
import gc
import time
import joblib
import os # Thêm os để tạo thư mục nếu chưa có

# Các thuật toán và công cụ đánh giá từ Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV # THÊM MỚI

# Thư viện XGBoost
import xgboost as xgb

# Vẽ biểu đồ
import matplotlib.pyplot as plt
import seaborn as sns

# Thư viện PyTorch (chỉ dùng để tải label_mapping nếu bạn đã lưu bằng torch.save)
import torch

# --- Hàm trợ giúp để đánh giá mô hình ---
# Hàm evaluate_model của bạn giữ nguyên (như bạn đã cung cấp)
def evaluate_model(model, X_test, y_test, class_names, model_name, results_dir="dataset/working1"):
    """
    Đánh giá mô hình, in các chỉ số, lưu kết quả và vẽ ma trận nhầm lẫn.
    """
    # Đảm bảo thư mục results_dir tồn tại
    os.makedirs(results_dir, exist_ok=True)
    
    print(f"\n🔍 Bắt đầu đánh giá mô hình: {model_name}...")
    start_eval_time = time.time()

    y_pred = model.predict(X_test)
    eval_time = time.time() - start_eval_time
    print(f"  - Thời gian dự đoán: {eval_time:.2f} giây")

    accuracy = accuracy_score(y_test, y_pred) * 100
    try:
        report = classification_report(y_test, y_pred, target_names=class_names, zero_division=0, digits=4)
    except ValueError as e:
        print(f"⚠️ Cảnh báo khi tạo classification report cho {model_name}: {e}")
        try:
            report = classification_report(y_test, y_pred, zero_division=0, digits=4)
        except Exception as report_err:
            print(f"❌ Không thể tạo classification report: {report_err}")
            report = "Không thể tạo báo cáo."

    print(f"\n🏆 Độ chính xác cuối cùng của {model_name}: {accuracy:.2f}%")
    print(f"\n📊 Báo cáo phân loại (Classification Report) của {model_name}:\n{report}")

    results_filename = os.path.join(results_dir, f"{model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}_results.txt")
    try:
        with open(results_filename, 'w', encoding='utf-8') as f:
            f.write(f"{model_name} - Final Accuracy: {accuracy:.2f}%\n\n")
            f.write("Classification Report:\n")
            f.write(report)
        print(f"✅ Kết quả đã được lưu vào tệp: '{results_filename}'")
    except Exception as e:
        print(f"❌ Lỗi khi lưu kết quả cho {model_name}: {e}")

    cm_filename = os.path.join(results_dir, f"{model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}_confusion_matrix.png")
    try:
        plt.figure(figsize=(18, 15)) # Tăng kích thước cho dễ đọc hơn
        conf_matrix = confusion_matrix(y_test, y_pred, labels=np.arange(len(class_names))) # Đảm bảo labels đúng thứ tự
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 8})
        plt.title(f'Ma trận nhầm lẫn (Confusion Matrix) - {model_name}', fontsize=16)
        plt.ylabel('Nhãn Thật (True Label)', fontsize=14)
        plt.xlabel('Nhãn Dự đoán (Predicted Label)', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(rotation=0, fontsize=10)
        plt.tight_layout()
        plt.savefig(cm_filename, dpi=150)
        print(f"📈 Ma trận nhầm lẫn đã được lưu vào tệp: '{cm_filename}'")
        plt.close()
    except Exception as e:
        print(f"❌ Lỗi khi tạo/lưu ma trận nhầm lẫn cho {model_name}: {e}")
        # traceback.print_exc() # Bỏ comment để xem chi tiết lỗi nếu cần

# --- Chương trình chính ---
start_total_time = time.time()
gc.collect()

# --- Thiết lập đường dẫn ---
data_dir = "dataset/working1" # Thư mục chứa dữ liệu đã xử lý của bạn
results_dir = os.path.join(data_dir, "tuning_results") # Thư mục mới để lưu kết quả tuning
os.makedirs(results_dir, exist_ok=True) # Tạo thư mục nếu chưa có

train_file = os.path.join(data_dir, "train_balanced1.csv") # File train đã qua SMOTE
test_file = os.path.join(data_dir, "test1.csv") # File test
label_map_file = os.path.join(data_dir, "label_mapping.pth") # File ánh xạ nhãn

# --- Tải dữ liệu ---
print("\n📂 Đang tải dữ liệu đã tiền xử lý...")
try:
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)
    print(f"  - Kích thước tập huấn luyện: {train_data.shape}")
    print(f"  - Kích thước tập kiểm thử: {test_data.shape}")
except FileNotFoundError as e:
    print(f"❌ Lỗi: Không tìm thấy tệp dữ liệu. {e}")
    exit()
except Exception as e:
    print(f"❌ Lỗi không xác định khi tải dữ liệu: {e}")
    exit()

# --- Tải bản đồ nhãn (Label Mapping) ---
print("\n🏷️ Đang tải bản đồ nhãn...")
try:
    label_mapping = torch.load(label_map_file)
    # Sắp xếp class_names dựa trên giá trị của mapping (0, 1, 2, ...)
    class_names = [k for k, v in sorted(label_mapping.items(), key=lambda item: item[1])]
    num_classes = len(class_names)
    print(f"  - Tải thành công bản đồ nhãn cho {num_classes} lớp.")
    print(f"  - Các lớp: {class_names}")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy tệp bản đồ nhãn '{label_map_file}'.")
    max_label_train = train_data['Label'].max() if 'Label' in train_data else -1
    max_label_test = test_data['Label'].max() if 'Label' in test_data else -1
    max_label = int(max(max_label_train, max_label_test))
    if max_label >= 0:
        class_names = [f"Lop_{i}" for i in range(max_label + 1)] # Cần đảm bảo thứ tự đúng
        num_classes = len(class_names)
        print(f"⚠️ Đang sử dụng tên lớp mặc định: {class_names}")
    else:
        print("❌ Không thể xác định số lớp từ dữ liệu.")
        exit()
except Exception as e:
    print(f"❌ Lỗi khi tải bản đồ nhãn: {e}")
    exit()

# --- Chuẩn bị dữ liệu cho mô hình ---
print("\n⚙️ Đang chuẩn bị dữ liệu (chuyển đổi sang NumPy)...")
try:
    X_train_np = train_data.drop(columns=["Label"]).values
    y_train_np = train_data["Label"].values
    X_test_np = test_data.drop(columns=["Label"]).values
    y_test_np = test_data["Label"].values
    print(f"  - Kích thước X_train: {X_train_np.shape}, y_train: {y_train_np.shape}")
    print(f"  - Kích thước X_test: {X_test_np.shape}, y_test: {y_test_np.shape}")
except KeyError as e:
    print(f"❌ Lỗi: Không tìm thấy cột 'Label' trong DataFrame. {e}")
    exit()
except Exception as e:
    print(f"❌ Lỗi khi chuẩn bị dữ liệu NumPy: {e}")
    exit()

print("\n🔥 Kiểm tra phạm vi giá trị dữ liệu (sau khi tải):")
min_train, max_train = X_train_np.min(), X_train_np.max()
min_test, max_test = X_test_np.min(), X_test_np.max()
print(f"  - Phạm vi đặc trưng Train: {min_train:.4f} đến {max_train:.4f}") # Đã chuẩn hóa nên thường là 0-1
print(f"  - Phạm vi đặc trưng Test: {min_test:.4f} đến {max_test:.4f}")   # Có thể >1 nếu test có giá trị ngoài khoảng train

del train_data, test_data
gc.collect()
print("  - Đã giải phóng bộ nhớ từ Pandas DataFrames.")


# --- CẤU HÌNH CHUNG CHO RandomizedSearchCV ---
# CHÚ Ý: TĂNG n_iter và cv để có kết quả tốt hơn, nhưng sẽ tốn nhiều thời gian hơn.
N_ITER_SEARCH = 5 # Số lượng kết hợp tham số được thử (TĂNG LÊN ít nhất 10-20)
CV_FOLDS = 2      # Số fold cho cross-validation (TĂNG LÊN 3 hoặc 5)
SCORING_METRIC = 'f1_weighted' # Metric để tối ưu

# --- 1. Tinh chỉnh siêu tham số và Huấn luyện Random Forest ---
print("\n" + "="*10 + " 🌳 Bắt đầu Tinh chỉnh và Huấn luyện Random Forest " + "="*10)

# Định nghĩa không gian siêu tham số cho RandomForest
rf_param_dist = {
    'n_estimators': [100, 150, 200], # Giảm bớt để chạy nhanh hơn, bạn có thể thêm 250, 300
    'max_depth': [20, 30, None], # None nghĩa là không giới hạn
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None], # None tương đương với tất cả các feature
    # 'class_weight': [None, 'balanced', 'balanced_subsample'] # SMOTE đã xử lý, nhưng có thể thử thêm
}

rf_base_model = RandomForestClassifier(random_state=42, n_jobs=-1)

rf_random_search = RandomizedSearchCV(
    estimator=rf_base_model,
    param_distributions=rf_param_dist,
    n_iter=N_ITER_SEARCH,
    cv=CV_FOLDS,
    scoring=SCORING_METRIC,
    verbose=2, # Hiển thị log quá trình tìm kiếm
    random_state=42,
    n_jobs=-1 # Sử dụng tất cả CPU cores cho cross-validation
)

print(f"\n⏳ Bắt đầu RandomizedSearchCV cho Random Forest (n_iter={N_ITER_SEARCH}, cv={CV_FOLDS})...")
start_rf_tuning_time = time.time()
try:
    rf_random_search.fit(X_train_np, y_train_np)
    rf_tuning_time = time.time() - start_rf_tuning_time
    print(f"✅ Hoàn thành RandomizedSearchCV cho Random Forest trong {rf_tuning_time:.2f} giây.")
    print(f"🌟 Siêu tham số tốt nhất cho Random Forest: {rf_random_search.best_params_}")
    print(f"🌟 Điểm {SCORING_METRIC} tốt nhất từ cross-validation: {rf_random_search.best_score_:.4f}")

    # Huấn luyện mô hình Random Forest cuối cùng với các tham số tốt nhất
    print("\n⚙️ Huấn luyện mô hình Random Forest cuối cùng với siêu tham số tốt nhất...")
    best_rf_model = rf_random_search.best_estimator_ # Lấy mô hình đã được fit với params tốt nhất trên toàn bộ X_train_np
    # Hoặc bạn có thể khởi tạo lại và fit:
    # best_rf_model = RandomForestClassifier(**rf_random_search.best_params_, random_state=42, n_jobs=-1)
    # best_rf_model.fit(X_train_np, y_train_np) # Nếu dùng best_estimator_ thì không cần fit lại

    # Lưu mô hình đã huấn luyện
    rf_model_path = os.path.join(results_dir, "tuned_random_forest_model.pkl")
    joblib.dump(best_rf_model, rf_model_path)
    print(f"💾 Mô hình Random Forest (đã tinh chỉnh) đã được lưu vào: '{rf_model_path}'")

    # Đánh giá mô hình
    evaluate_model(best_rf_model, X_test_np, y_test_np, class_names, "Random Forest (Tuned)", results_dir=results_dir)

except MemoryError:
    print(f"❌ Lỗi bộ nhớ khi tinh chỉnh/huấn luyện Random Forest.")
except Exception as e:
    print(f"❌ Đã xảy ra lỗi trong quá trình xử lý Random Forest: {e}")
    # import traceback
    # traceback.print_exc()

del rf_base_model, rf_random_search
if 'best_rf_model' in locals(): del best_rf_model
gc.collect()


# --- 2. Tinh chỉnh siêu tham số và Huấn luyện XGBoost ---
print("\n" + "="*10 + " 🚀 Bắt đầu Tinh chỉnh và Huấn luyện XGBoost " + "="*10)

# Kiểm tra GPU và thiết lập tree_method
xgb_tree_method = 'hist' # Mặc định là CPU
if torch.cuda.is_available():
    print("  - Phát hiện GPU NVIDIA và CUDA.")
    xgb_tree_method = 'hist' # Sử dụng 'hist' và sẽ đặt device='cuda'
    print(f"  - Sẽ sử dụng tree_method='{xgb_tree_method}' và device='cuda' cho XGBoost.")
else:
    print("  - Không phát hiện GPU NVIDIA hoặc CUDA.")
    print(f"  - Sẽ sử dụng tree_method='{xgb_tree_method}' (CPU) cho XGBoost.")

# Định nghĩa không gian siêu tham số cho XGBoost
xgb_param_dist = {
    'n_estimators': [100, 150, 200], # Giảm bớt để chạy nhanh hơn
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [6, 8, 10], # Giữ ở mức vừa phải
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5]
    # 'scale_pos_weight': # Có thể hữu ích nếu vẫn còn mất cân bằng đáng kể sau SMOTE cho một số lớp,
                          # nhưng XGBoost xử lý đa lớp, nên cần cẩn thận.
                          # Với SMOTE, thường không cần thiết bằng.
}

# Các tham số cố định cho XGBoost
xgb_fixed_params = {
    'objective': 'multi:softmax',
    'num_class': num_classes,
    'random_state': 42,
    'eval_metric': 'mlogloss', # Hoặc 'merror'
    'n_jobs': -1, # XGBoost tự quản lý threads, n_jobs của RandomizedSearchCV sẽ lo CV
    'tree_method': xgb_tree_method
}
if xgb_tree_method == 'hist' and torch.cuda.is_available(): # Sửa lại theo khuyến cáo mới
    xgb_fixed_params['device'] = 'cuda'
    # Nếu dùng phiên bản XGBoost cũ hơn và 'gpu_hist' vẫn hoạt động, bạn có thể dùng:
    # xgb_fixed_params['tree_method'] = 'gpu_hist'
else:
    xgb_fixed_params['tree_method'] = 'hist' # Đảm bảo là 'hist' cho CPU

xgb_base_model = xgb.XGBClassifier(**xgb_fixed_params)

xgb_random_search = RandomizedSearchCV(
    estimator=xgb_base_model,
    param_distributions=xgb_param_dist,
    n_iter=N_ITER_SEARCH,
    cv=CV_FOLDS,
    scoring=SCORING_METRIC,
    verbose=2,
    random_state=42,
    n_jobs=-1 # n_jobs của RandomizedSearchCV
)

print(f"\n⏳ Bắt đầu RandomizedSearchCV cho XGBoost (n_iter={N_ITER_SEARCH}, cv={CV_FOLDS})...")
start_xgb_tuning_time = time.time()
try:
    xgb_random_search.fit(X_train_np, y_train_np)
    xgb_tuning_time = time.time() - start_xgb_tuning_time
    print(f"✅ Hoàn thành RandomizedSearchCV cho XGBoost trong {xgb_tuning_time:.2f} giây.")
    print(f"🌟 Siêu tham số tốt nhất cho XGBoost: {xgb_random_search.best_params_}")
    print(f"🌟 Điểm {SCORING_METRIC} tốt nhất từ cross-validation: {xgb_random_search.best_score_:.4f}")

    # Huấn luyện mô hình XGBoost cuối cùng với các tham số tốt nhất
    print("\n⚙️ Huấn luyện mô hình XGBoost cuối cùng với siêu tham số tốt nhất...")
    # Kết hợp tham số cố định và tham số tốt nhất từ tìm kiếm
    final_xgb_params = {**xgb_fixed_params, **xgb_random_search.best_params_}
    best_xgb_model = xgb.XGBClassifier(**final_xgb_params)
    
    # Kiểm tra lại device nếu cần thiết trước khi fit
    if 'device' in final_xgb_params and final_xgb_params['device'] == 'cuda' and not torch.cuda.is_available():
        print("⚠️ CUDA không khả dụng, chuyển XGBoost sang CPU.")
        final_xgb_params['device'] = 'cpu'
        final_xgb_params['tree_method'] = 'hist'
        best_xgb_model = xgb.XGBClassifier(**final_xgb_params) # Khởi tạo lại với params CPU

    best_xgb_model.fit(X_train_np, y_train_np, verbose=False)

    # Lưu mô hình
    xgb_model_path = os.path.join(results_dir, "tuned_xgboost_model.pkl")
    # Kiểm tra xem model có thuộc tính device không (phiên bản XGBoost mới)
    if hasattr(best_xgb_model, 'device') and best_xgb_model.device == 'cuda:0':
         # Chuyển model về CPU trước khi lưu bằng joblib nếu nó đang ở trên GPU
         # Điều này giúp tránh lỗi khi tải model trên môi trường không có GPU
        try:
            print("  - Chuyển model XGBoost về CPU trước khi lưu...")
            best_xgb_model.set_params(device='cpu') # Thử cách này
            # Hoặc nếu cách trên không được, bạn có thể cần get_params, xóa device, rồi tạo lại
            # params = best_xgb_model.get_params()
            # params.pop('device', None) # Bỏ device
            # params.pop('tree_method', None) # Bỏ tree_method để nó tự chọn hist
            # cpu_model = xgb.XGBClassifier(**params)
            # cpu_model.fit(X_train_np, y_train_np) # Fit lại trên CPU, hơi tốn tgian
            # joblib.dump(cpu_model, xgb_model_path)
            joblib.dump(best_xgb_model, xgb_model_path) # Lưu model đã chuyển về CPU (nếu set_params thành công)

        except Exception as e_cpu:
            print(f"  - ⚠️ Không thể chuyển model XGBoost về CPU, lưu trực tiếp: {e_cpu}")
            joblib.dump(best_xgb_model, xgb_model_path) # Vẫn cố lưu
    else:
        joblib.dump(best_xgb_model, xgb_model_path) # Lưu model

    print(f"💾 Mô hình XGBoost (đã tinh chỉnh) đã được lưu vào: '{xgb_model_path}'")

    # Đánh giá mô hình
    evaluate_model(best_xgb_model, X_test_np, y_test_np, class_names, "XGBoost (Tuned)", results_dir=results_dir)

except xgb.core.XGBoostError as e:
    print(f"❌ Lỗi XGBoost: {e}")
    if "CUDA" in str(e) or "GPU" in str(e):
        print("  - Lỗi có thể liên quan đến cài đặt CUDA hoặc GPU.")
except MemoryError:
    print(f"❌ Lỗi bộ nhớ khi tinh chỉnh/huấn luyện XGBoost.")
except Exception as e:
    print(f"❌ Đã xảy ra lỗi trong quá trình xử lý XGBoost: {e}")
    # import traceback
    # traceback.print_exc()


# --- Hoàn tất ---
del X_train_np, y_train_np, X_test_np, y_test_np
if 'xgb_base_model' in locals(): del xgb_base_model
if 'xgb_random_search' in locals(): del xgb_random_search
if 'best_xgb_model' in locals(): del best_xgb_model
gc.collect()

end_total_time = time.time()
total_script_time = end_total_time - start_total_time
print(f"\n🏁🏁🏁 Quá trình tinh chỉnh, huấn luyện và đánh giá hoàn tất trong {total_script_time:.2f} giây! 🏁🏁🏁")


📂 Đang tải dữ liệu đã tiền xử lý...
  - Kích thước tập huấn luyện: (4719765, 73)
  - Kích thước tập kiểm thử: (1176365, 73)

🏷️ Đang tải bản đồ nhãn...
  - Tải thành công bản đồ nhãn cho 14 lớp.
  - Các lớp: ['Benign', 'Bot', 'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP', 'DoS attacks-GoldenEye', 'DoS attacks-Hulk', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Slowloris', 'FTP-BruteForce', 'Infilteration', 'SSH-Bruteforce', 'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection']

⚙️ Đang chuẩn bị dữ liệu (chuyển đổi sang NumPy)...
  - Kích thước X_train: (4719765, 72), y_train: (4719765,)
  - Kích thước X_test: (1176365, 72), y_test: (1176365,)

🔥 Kiểm tra phạm vi giá trị dữ liệu (sau khi tải):
  - Phạm vi đặc trưng Train: 0.0000 đến 1.0000
  - Phạm vi đặc trưng Test: 0.0000 đến 3.4572
  - Đã giải phóng bộ nhớ từ Pandas DataFrames.

========== 🌳 Bắt đầu Tinh chỉnh và Huấn luyện Random Forest ==========

⏳ Bắt đầu RandomizedSearchCV cho Random Forest (n_iter=5, cv=2)...
Fitting 2 folds for e

d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
6 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Do_an_tot_nghiep\apt-detection\venv\Lib\site-packages\sklearn\ensemb

KeyboardInterrupt: 